# Lecture02 PC hardware and x86 programming slides

**Outline**

* PC architecture
* x86 instruction set
* gcc calling conventions
* PC emulation

## PC architecture

A full PC has:

* an x86 CPU with registers, execution unit and memory management
* CPU chip pins include address and data signals
* memory which holds instructions and data
* disk
* keyboard
* display
* other resources: BIOS ROM, clock, ...

Abstract model:

* I/O: communicating data to and from devices
* CPU: digital logic for performing computation
* Memory: N words of B bits

![abstract model](images/figure02-01.png)

Needs work space: **registers**

* four 16-bit data registers: AX, BX, CX, DX
* each in two 8-bit halves, e.g. AX -> AH and AL
* very fast, very few

More work space: **memory**

* CPU sends out address on address lines (wires, one bit per wire)
* Data comes back on data lines or data is written to data lines

Add address registers: **pointers into memory**

* SP - stack pointer
* BP - frame base pointer
* SI - source index
* DI - destination index

Instructions are in memory: **IP - instruction pointer**, it will increse after running each instruction and can be modified by CALL, RET, JMP, conditional jumps.

Want conditional jumps: **FLAGS - various condition codes**

![FLAGS](images/figure02-02.png)

Interact with outside world: **I/O**

* Original PC architecture: use dedicated I/O space
    * works same as memory accesses but set I/O signal
    * only 1024 I/O addresses
    * accessed with special instructions
* Memory-Mapped I/O
    * use normal physical memory addresses
        * No need for special instructions
        * No 1024 limit
        * system controller routes to device
    * works like `magic` memory
        * addressed and accessed like memory
        * but does not behave like memory
        * reads and writes have "side effects"
        * read result can change due to external events

More memory:
* 8086, 16 registers and 20-bit bus addresses
* 80386, 32 bit data and bus addresses
* 80386 boots in 16-bit mode, and boot.S switches to protected mode with 32-bit addresses
* 80386 also added virtual memory addresses
    * Segment registers are indices into a table
    * Page table hardware

## x86 Physical Memory Map

* The physical address space mostly looks like ordinary RAM
* Except some low-memory address actually refer to other things
* Writes to VGA memory appear on the screen
* Reset or power on jump to ROM at 0xfffffff0

```
+------------------+  <- 0xFFFFFFFF (4GB)
|      32-bit      |
|  memory mapped   |
|     devices      |
|                  |
/\/\/\/\/\/\/\/\/\/\

/\/\/\/\/\/\/\/\/\/\
|                  |
|      Unused      |
|                  |
+------------------+  <- depends on amount of RAM
|                  |
|                  |
| Extended Memory  |
|                  |
|                  |
+------------------+  <- 0x00100000 (1MB)
|     BIOS ROM     |
+------------------+  <- 0x000F0000 (960KB)
|  16-bit devices, |
|  expansion ROMs  |
+------------------+  <- 0x000C0000 (768KB)
|   VGA Display    |
+------------------+  <- 0x000A0000 (640KB)
|                  |
|    Low Memory    |
|                  |
+------------------+  <- 0x00000000
```

## x86 Instruction Set

Instructions classes:
* Data movement: MOV, PUSH, POP, ...
* Arithmetic: TEST, SHL, ADD, ...
* I/O: IN, OUT, ...
* Control: JMP, JZ, JNZ, CALL, RET
* String: REP, MOVSB, ...
* System: IRET, INT, ...

## Gcc calling conventions for JOS

Each function has a stack frame marked by %ebp, %esp:

```
               +------------+   |
		       | arg 2      |   \
		       +------------+    >- previous function's stack frame
		       | arg 1      |   /
		       +------------+   |
		       | ret %eip   |   /
		       +============+   
		       | saved %ebp |   \
		%ebp-> +------------+   |
		       |            |   |
		       |   local    |   \
		       | variables, |    >- current function's stack frame
		       |    etc.    |   /
		       |            |   |
		       |            |   |
		%esp-> +------------+   /
```

Compiling, linking, loading:
* Preprocessor takes C source code (ASCII text), expands #include etc, produces C source code
* Compiler takes C source code (ASCII text), produces assembly language (also ASCII text)
* Assembler takes assembly language (ASCII text), produces .o file (binary, machine-readable!)
* Linker takes multiple '.o's, produces a single program image (binary)
* Loader loads the program image into memory at run-time and starts it executing

![Compiling, linking, loading](images/figure02-03.png)

## PC emulation

QEMU PC emulator:
* does what a real PC does
* only implemented in software

Emulating devices:
* Hard dis: using a file of the host
* VGA display: draw in a host window
* Key board: hosts's key board API
* Clock chip: host's clock